# Implement the data  Processing , data visualization and data wrangling on any real  world problem i.e. Covid_19 dataset to view the active cases on the world map using choropleth and also plot the cases

##### geospatial data visulization and analysis using folium library
##### COVID_19 data is accessed through an api
##### coordinates for indian states are traced using GeoJSON

In [9]:
pip install folium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import folium as file
from folium import plugins
import pandas as pd
import json
import requests

In [11]:
with open(r'Indian_States.json') as f:
    geojson_states = json.load(f)
    

In [12]:
for i in geojson_states['features']:
    i['id'] = i['properties']['NAME_1']

In [15]:
with open("data.json","r") as file:
    covid_current = json.load(file)

In [18]:
import pandas as pd
data_list = []
for state_data in covid_current["statewise"][1:]:
    if state_data["state"]!= "State Unassigned":
        data_list.append({
            "State": state_data["state"],
            "Total Cases": int(state_data["confirmed"]),
            "Active Cases": int(state_data["active"]),
            "Death": int(state_data["deaths"])
        })
        df_covid=pd.DataFrame(data_list)

In [19]:
df_covid.rename(columns={
    "Total Cases": "total_case",
    "Active Cases": "active_cases",
    "Deaths": "total_deaths"
},inplace=True)

In [27]:
geojson_states_names = [feature["properties"]["NAME_1"]  for feature in geojson_states["features"]]
df_covid_state = df_covid["State"].unique()
geojson_state_names = [feature["properties"]["NAME_1"] for feature in geojson_states["features"]]
df_covid_states = df_covid["State"].unique()
missing_states = set(geojson_states_names) - set(df_covid_states)
print("State present in GeoJSON but missing in DataFrame:", missing_states)
state_name_corrections ={
    "Dadar and  Nagar Haveli and Daman and Diu": "Dadar and Nagar Haveli",
    "NCT of Delhi": "Delhi"
}
df_covid["State"] = df_covid["State"].replace(state_name_corrections)
state_name_corrections ={
    "Dadar and Nagar Haveli and Daman and Diu":"Dadar and Nagar Haveli",
    "NCT if Delhi":"Delhi"
}
df_covid["State"] = df_covid["State"].replace(state_name_corrections)

State present in GeoJSON but missing in DataFrame: {'Dadra and Nagar Haveli'}


In [ ]:
state_data_map = df_covid.set_index("State").to_dict(orient="index")
for feature in geojson_states["features"]:
    states_name = feature["properties"]["NAME_1"]
    if state_name in state_data_map:
        feature["properties"]["total_case"] = state_data_map[state_name]["total_case"]
        feature["properties"]["active_case"] = state_data_map[state_name]["active_case"]
        feature["properties"]["total_deaths"] = state_data_map[state_name]["total_deaths"]
    else:
        feature["properties"]["total_case"] = "No Data